# QUESTION 1 : Parametric Algorithms 

## a. Model Design Under Constraints

In [5]:
import pandas as pd
import numpy as np
import os
os.chdir(r"C:\Users\raksh\Documents\MBA FBM\3rd Semester\Application of Softwares")
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [6]:
data = pd.read_csv("weekly_sales_dataset.csv")

In [8]:
TARGET = "Weekly_Sales"

X = data.drop(columns=[TARGET])
y = data[TARGET]
print (
"""
Chosen Model: Multiple Linear Regression (Parametric)

Justification:
- Coefficients are directly interpretable (₹ impact per unit change)
- Easy to explain during business reviews
- Low variance → stable over long update cycles
- Transparent risk assessment for budget allocation

Why NOT other parametric models?
- Polynomial regression → unstable extrapolation
- Lasso → feature elimination may confuse stakeholders
- Time-series ARIMA → harder to explain, less feature-driven
"""
)
model = LinearRegression()


Chosen Model: Multiple Linear Regression (Parametric)

Justification:
- Coefficients are directly interpretable (₹ impact per unit change)
- Easy to explain during business reviews
- Low variance → stable over long update cycles
- Transparent risk assessment for budget allocation

Why NOT other parametric models?
- Polynomial regression → unstable extrapolation
- Lasso → feature elimination may confuse stakeholders
- Time-series ARIMA → harder to explain, less feature-driven



### Why Multiple Linear Regression ? My understanding with conceptual data

In [92]:
multi_lr = LinearRegression()
multi_lr.fit(X, y)

multi_preds = multi_lr.predict(X)
multi_mae = mean_absolute_error(y, multi_preds)

print("Multiple Linear Regression MAE:", multi_mae)


Multiple Linear Regression MAE: 96.31604096238722


In [93]:
coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": multi_lr.coef_
}).sort_values(by="Coefficient", ascending=False)

coef_df


,Feature,Coefficient
2,Competitor_Price,13.372099
0,Advertising_Spend,1.899227
1,Price,-20.124950


In [97]:
tscv = TimeSeriesSplit(n_splits=5)
TARGET = "Weekly_Sales"
PRIMARY_FEATURE = "Advertising_Spend"

simple_mae_cv = []
multi_mae_cv = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Simple LR
    slr = LinearRegression()
    slr.fit(X_train[[PRIMARY_FEATURE]], y_train)
    simple_mae_cv.append(
        mean_absolute_error(
            y_test,
            slr.predict(X_test[[PRIMARY_FEATURE]])
        )
    )
    
    # Multiple LR
    mlr = LinearRegression()
    mlr.fit(X_train, y_train)
    multi_mae_cv.append(
        mean_absolute_error(
            y_test,
            mlr.predict(X_test)
        )
    )

print("\nSimple LR CV MAE :", np.mean(simple_mae_cv))
print("Multiple LR CV MAE:", np.mean(multi_mae_cv))



Simple LR CV MAE : 187.61240106809325
Multiple LR CV MAE: 99.43916515970898


In [98]:
print("Simple LR MAE Variance :", np.var(simple_mae_cv))
print("Multiple LR MAE Variance:", np.var(multi_mae_cv))


Simple LR MAE Variance : 52.28238041463786
Multiple LR MAE Variance: 100.06564331962379


In [99]:
print("\nSimple LR MAE Variance :", np.var(simple_mae_cv))
print("Multiple LR MAE Variance:", np.var(multi_mae_cv))



Simple LR MAE Variance : 52.28238041463786
Multiple LR MAE Variance: 100.06564331962379


In [101]:
# --- SAFETY RECOMPUTATION (prevents NameError) ---

# Simple Linear Regression (recompute MAE)
simple_lr = LinearRegression()
simple_lr.fit(X[[PRIMARY_FEATURE]], y)
simple_preds = simple_lr.predict(X[[PRIMARY_FEATURE]])
simple_mae = mean_absolute_error(y, simple_preds)

# Multiple Linear Regression (recompute MAE)
multi_lr = LinearRegression()
multi_lr.fit(X, y)
multi_preds = multi_lr.predict(X)
multi_mae = mean_absolute_error(y, multi_preds)

# Time-based CV recomputation
tscv = TimeSeriesSplit(n_splits=5)

simple_mae_cv = []
multi_mae_cv = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    slr = LinearRegression()
    slr.fit(X_train[[PRIMARY_FEATURE]], y_train)
    simple_mae_cv.append(
        mean_absolute_error(y_test, slr.predict(X_test[[PRIMARY_FEATURE]]))
    )

    mlr = LinearRegression()
    mlr.fit(X_train, y_train)
    multi_mae_cv.append(
        mean_absolute_error(y_test, mlr.predict(X_test))
    )

# --- FINAL COMPARISON TABLE ---

comparison = pd.DataFrame({
    "Aspect": [
        "Number of predictors",
        "Interpretability",
        "Captures multiple business drivers",
        "In-sample MAE",
        "Time-based CV MAE",
        "Stability for quarterly updates",
        "Suitability for budget allocation"
    ],
    "Simple Linear Regression": [
        "1",
        "Very High",
        "No",
        simple_mae,
        np.mean(simple_mae_cv),
        "High",
        "Limited"
    ],
    "Multiple Linear Regression": [
        "Multiple",
        "High",
        "Yes",
        multi_mae,
        np.mean(multi_mae_cv),
        "High",
        "Strong"
    ]
})

comparison


,Aspect,Simple Linear Regression,Multiple Linear Regression
0,Number of predictors,1,Multiple
1,Interpretability,Very High,High
2,Captures multiple business drivers,No,Yes
3,In-sample MAE,179.854439,96.316041
4,Time-based CV MAE,187.612401,99.439165
5,Stability for quarterly updates,High,High
6,Suitability for budget allocation,Limited,Strong


### Modelling Choices

In [9]:
print(
"""
Modeling Choice 1: No Polynomial or Interaction Terms
Reason:
- Linear effects are easier to explain to business teams
- Reduces overfitting → safer for quarterly updates
- Slight accuracy loss is acceptable for budget stability
"""

"""
Modeling Choice 2: Limited Feature Set (No Aggressive Feature Engineering)
Reason:
- Fewer drivers → easier accountability
- Prevents sudden coefficient swings across quarters
- Stakeholders trust simple cause–effect relationships
"""
)


Modeling Choice 1: No Polynomial or Interaction Terms
Reason:
- Linear effects are easier to explain to business teams
- Reduces overfitting → safer for quarterly updates
- Slight accuracy loss is acceptable for budget stability

Modeling Choice 2: Limited Feature Set (No Aggressive Feature Engineering)
Reason:
- Fewer drivers → easier accountability
- Prevents sudden coefficient swings across quarters
- Stakeholders trust simple cause–effect relationships



In [41]:
print("""
PRIORITIZED DIAGNOSTIC:
Mean Absolute Error (MAE)

Why?
- Directly interpretable in ₹ terms
- Aligns with budget planning risk
- Penalizes errors linearly (no exaggeration)
"""

"""
DEPRIORITIZED DIAGNOSTIC:
R² Score

Why?
- High R² does not guarantee good forecasts
- Can mislead non-technical stakeholders
- Budget decisions care about error magnitude, not variance explained
"""
"""
Deployment Strategy:
- Freeze coefficients for the entire quarter
- Monitor MAE weekly (no retraining)
- Full retraining only at quarter-end
- Any coefficient drift explained in business terms
"""
     )


PRIORITIZED DIAGNOSTIC:
Mean Absolute Error (MAE)

Why?
- Directly interpretable in ₹ terms
- Aligns with budget planning risk
- Penalizes errors linearly (no exaggeration)

DEPRIORITIZED DIAGNOSTIC:
R² Score

Why?
- High R² does not guarantee good forecasts
- Can mislead non-technical stakeholders
- Budget decisions care about error magnitude, not variance explained

Deployment Strategy:
- Freeze coefficients for the entire quarter
- Monitor MAE weekly (no retraining)
- Full retraining only at quarter-end
- Any coefficient drift explained in business terms



## Support Evidence

In [58]:
lr = LinearRegression()
lr.fit(X, y)

coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": lr.coef_
}).sort_values(by="Coefficient", ascending=False)

coef_df


,Feature,Coefficient
2,Competitor_Price,13.372099
0,Advertising_Spend,1.899227
1,Price,-20.124950


In [59]:
tscv = TimeSeriesSplit(n_splits=5)
coef_history = []

for train_idx, test_idx in tscv.split(X):
    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]
    
    temp_model = LinearRegression()
    temp_model.fit(X_train, y_train)
    
    coef_history.append(temp_model.coef_)

coef_stability = pd.DataFrame(coef_history, columns=X.columns)
coef_stability


,Advertising_Spend,Price,Competitor_Price
0,1.872941,-17.978115,17.209283
1,1.860166,-19.521379,14.589344
2,1.857569,-19.578647,13.917004
3,1.857040,-19.176156,14.276822
4,1.904804,-19.902301,14.250083


In [60]:
mae_scores = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    mae_scores.append(mean_absolute_error(y_test, preds))

np.mean(mae_scores)


99.43916515970898

In [61]:
from sklearn.metrics import r2_score

r2_scores = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    r2_scores.append(r2_score(y_test, preds))

np.mean(r2_scores)


0.8359535423862627

In [62]:
corr_matrix = X.corr()
corr_matrix
corr_matrix.abs().max()


Advertising_Spend    1.0
Price                1.0
Competitor_Price     1.0
dtype: float64

In [63]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_const = sm.add_constant(X)

vif_df = pd.DataFrame({
    "Feature": X_const.columns,
    "VIF": [variance_inflation_factor(X_const.values, i)
            for i in range(X_const.shape[1])]
})

vif_df


,Feature,VIF
0,const,126.355904
1,Advertising_Spend,1.003182
2,Price,1.002726
3,Competitor_Price,1.002515


In [64]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

poly_model = Pipeline([
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("lr", LinearRegression())
])

poly_mae = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    poly_model.fit(X_train, y_train)
    preds = poly_model.predict(X_test)
    
    poly_mae.append(mean_absolute_error(y_test, preds))

np.mean(poly_mae)


104.51887429413705

In [65]:
print("""
evidence confirms that:

1. Linear Regression provides stable, interpretable coefficients.
2. MAE is the most business-aligned validation metric.
3. Multicollinearity is present but not severe.
4. More complex models offer limited accuracy gains
   while increasing governance and decision risk.

Therefore, Linear Regression is the safest deployable
parametric model under the given constraints.
"""
     )


evidence confirms that:

1. Linear Regression provides stable, interpretable coefficients.
2. MAE is the most business-aligned validation metric.
3. Multicollinearity is present but not severe.
4. More complex models offer limited accuracy gains
   while increasing governance and decision risk.

Therefore, Linear Regression is the safest deployable
parametric model under the given constraints.



## b. Model Risk and Counterfactual Analysis

In [17]:
TARGET = "Weekly_Sales"
AD_FEATURE = "Advertising_Spend" 

X = data.drop(columns=[TARGET])
y = data[TARGET]

In [18]:
tscv = TimeSeriesSplit(n_splits=5)

mae_scores = []
rmse_scores = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = LinearRegression()
    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    mae_scores.append(mean_absolute_error(y_test, preds))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, preds)))

print("Baseline Model Performance")
print("Average MAE :", np.mean(mae_scores))
print("Average RMSE:", np.mean(rmse_scores))


Baseline Model Performance
Average MAE : 99.43916515970898
Average RMSE: 125.54942913294667


In [19]:
baseline_model = LinearRegression()
baseline_model.fit(X, y)

LinearRegression()

In [20]:
# Copy original features
X_cf = X.copy()

# Increase advertising spend by 20%
X_cf[AD_FEATURE] = X_cf[AD_FEATURE] * 1.20


In [21]:

y_pred_baseline = baseline_model.predict(X)
y_pred_cf = baseline_model.predict(X_cf)
delta_sales = y_pred_cf - y_pred_baseline


In [22]:
print("Average increase in predicted weekly sales:",
      np.mean(delta_sales))


Average increase in predicted weekly sales: 189.6698471627702


In [23]:
ad_coef = pd.Series(
    baseline_model.coef_,
    index=X.columns
)[AD_FEATURE]

print("Advertising coefficient:", ad_coef)


Advertising coefficient: 1.8992269758081617


In [24]:
avg_ad_spend = X[AD_FEATURE].mean()
avg_sales_lift = np.mean(delta_sales)

roi_ratio = avg_sales_lift / (0.20 * avg_ad_spend)

print("Sales increase per ₹1 additional ad spend:", roi_ratio)


Sales increase per ₹1 additional ad spend: 1.8992269758081612


In [25]:
print("""
Model Risk Assessment:

1. Linearity Assumption:
   The model assumes advertising has a constant marginal impact.
   At high spend levels, this may overestimate returns.

2. No Saturation Effects:
   Real-world advertising often exhibits diminishing returns.
   This counterfactual should be interpreted as short-term impact only.

3. Controlled Change:
   Holding other variables constant isolates the effect,
   but real business environments may show interaction effects.
"""
     )


Model Risk Assessment:

1. Linearity Assumption:
   The model assumes advertising has a constant marginal impact.
   At high spend levels, this may overestimate returns.

2. No Saturation Effects:
   Real-world advertising often exhibits diminishing returns.
   This counterfactual should be interpreted as short-term impact only.

3. Controlled Change:
   Holding other variables constant isolates the effect,
   but real business environments may show interaction effects.



In [26]:
TARGET = "Weekly_Sales"
KEY_VAR = "Advertising_spend"

X = data.drop(columns=[TARGET])
y = data[TARGET]

# Train correct baseline model
baseline_model = LinearRegression()
baseline_model.fit(X, y)

baseline_preds = baseline_model.predict(X)
baseline_mae = mean_absolute_error(y, baseline_preds)

print("Baseline MAE:", baseline_mae)


Baseline MAE: 96.31604096238722


In [31]:

KEY_VAR = "Advertising_Spend"
print(X.columns.tolist())


['Advertising_Spend', 'Price', 'Competitor_Price']


In [32]:
X_miss = X.drop(columns=[KEY_VAR])

miss_model = LinearRegression()
miss_model.fit(X_miss, y)

miss_preds = miss_model.predict(X_miss)
miss_mae = mean_absolute_error(y, miss_preds)

print("Misspecified MAE:", miss_mae)

Misspecified MAE: 205.82579660836728


In [33]:
coef_comparison = pd.DataFrame({
    "Baseline": baseline_model.coef_,
    "Misspecified": np.append(miss_model.coef_, np.nan)
}, index=X.columns)

coef_comparison


,Baseline,Misspecified
Advertising_Spend,1.899227,-21.34195
Price,-20.124950,12.11365
Competitor_Price,13.372099,NaN


In [34]:
prediction_shift = miss_preds - baseline_preds

print("Average change in predictions due to misspecification:",
      np.mean(prediction_shift))


Average change in predictions due to misspecification: -8.943364567433794e-14


In [35]:
np.random.seed(42)

X_noisy = X.copy()
noise = np.random.normal(0, 0.10, X.shape)  # 10% noise
X_noisy = X_noisy * (1 + noise)


In [36]:
noisy_model = LinearRegression()
noisy_model.fit(X_noisy, y)

noisy_preds = noisy_model.predict(X_noisy)
noisy_mae = mean_absolute_error(y, noisy_preds)

print("Noisy Model MAE:", noisy_mae)


Noisy Model MAE: 142.89918868226624


In [37]:
coef_noise = pd.DataFrame({
    "Baseline": baseline_model.coef_,
    "Noisy": noisy_model.coef_
}, index=X.columns)

coef_noise


,Baseline,Noisy
Advertising_Spend,1.899227,1.676872
Price,-20.124950,-12.872777
Competitor_Price,13.372099,9.008225


In [38]:
print("""
Business Risk from Misspecification:

1. Omitted variable bias leads to incorrect attribution of sales drivers.
2. Budget decisions may shift funds away from truly effective channels.
3. Noise-induced instability reduces trust in coefficients.
4. Quarterly model freezes amplify the cost of bad specifications.
"""
     )


Business Risk from Misspecification:

1. Omitted variable bias leads to incorrect attribution of sales drivers.
2. Budget decisions may shift funds away from truly effective channels.
3. Noise-induced instability reduces trust in coefficients.
4. Quarterly model freezes amplify the cost of bad specifications.



In [39]:
print("""
Why deployment should be refused:

1. Historical relationships no longer represent future reality.
2. Linear coefficients encode outdated causal mechanisms.
3. Counterfactual predictions become misleading.
4. Budget allocation based on old regimes causes financial loss.
"""
     )


Why deployment should be refused:

1. Historical relationships no longer represent future reality.
2. Linear coefficients encode outdated causal mechanisms.
3. Counterfactual predictions become misleading.
4. Budget allocation based on old regimes causes financial loss.



In [48]:
# Example performance metrics
mae = 1200       
rmse = 1800       
metrics_good = (mae < 1500) and (rmse < 2000)
print("Metrics acceptable:", metrics_good)


Metrics acceptable: True


In [43]:
structural_break_detected = True     
policy_change = False                 
channel_shift = True                  
data_definition_changed = False       

In [44]:
def deployment_decision(metrics_good,
                        structural_break_detected,
                        policy_change,
                        channel_shift,
                        data_definition_changed):
    
    if not metrics_good:
        return "❌ DO NOT DEPLOY: Model performance is insufficient"
    
    if (structural_break_detected or
        policy_change or
        channel_shift or
        data_definition_changed):
        
        return (
            "❌ DO NOT DEPLOY: Business regime change detected.\n"
            "Historical linear relationships are no longer valid.\n"
            "Model metrics are misleading for future decisions."
        )
    
    return "✅ DEPLOY MODEL: Metrics and business conditions are aligned"


In [45]:
decision = deployment_decision(
    metrics_good=metrics_good,
    structural_break_detected=structural_break_detected,
    policy_change=policy_change,
    channel_shift=channel_shift,
    data_definition_changed=data_definition_changed
)

print(decision)


❌ DO NOT DEPLOY: Business regime change detected.
Historical linear relationships are no longer valid.
Model metrics are misleading for future decisions.


In [47]:
print("""
Key Principle:
Model deployment is a BUSINESS decision, not a metric decision.

Even with strong MAE/RMSE:
- Structural breaks invalidate coefficients
- Counterfactuals become unsafe
- Budget allocation decisions become risky

Therefore, deployment must be refused.
"""
     )


Key Principle:
Model deployment is a BUSINESS decision, not a metric decision.

Even with strong MAE/RMSE:
- Structural breaks invalidate coefficients
- Counterfactuals become unsafe
- Budget allocation decisions become risky

Therefore, deployment must be refused.



# Question 2 : Non Parametric Algorithms

## a. Algorithmic Accountablitity

In [51]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit


In [49]:
TARGET = "Weekly_Sales"
AD_COL = "Advertising_Spend"

X = data.drop(columns=[TARGET])
y = data[TARGET]

In [52]:
tscv = TimeSeriesSplit(n_splits=5)

lr_mae = []
rf_mae = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    lr = LinearRegression()
    rf = RandomForestRegressor(n_estimators=200, random_state=42)

    lr.fit(X_train, y_train)
    rf.fit(X_train, y_train)

    lr_mae.append(mean_absolute_error(y_test, lr.predict(X_test)))
    rf_mae.append(mean_absolute_error(y_test, rf.predict(X_test)))

print("Linear Regression MAE:", np.mean(lr_mae))
print("Random Forest MAE    :", np.mean(rf_mae))


Linear Regression MAE: 99.43916515970898
Random Forest MAE    : 124.04444679999997


In [53]:
# Train final models
lr_final = LinearRegression()
rf_final = RandomForestRegressor(n_estimators=200, random_state=42)

lr_final.fit(X, y)
rf_final.fit(X, y)

# Create counterfactual
X_cf = X.copy()
X_cf[AD_COL] = X_cf[AD_COL] * 1.20

# Predictions
lr_base = lr_final.predict(X)
lr_cf = lr_final.predict(X_cf)

rf_base = rf_final.predict(X)
rf_cf = rf_final.predict(X_cf)

# Changes
lr_delta = lr_cf - lr_base
rf_delta = rf_cf - rf_base


In [54]:
print("Linear Regression – Avg Sales Change:", np.mean(lr_delta))
print("Random Forest – Avg Sales Change   :", np.mean(rf_delta))

print("\nLinear Regression – Std Dev:", np.std(lr_delta))
print("Random Forest – Std Dev   :", np.std(rf_delta))


Linear Regression – Avg Sales Change: 189.6698471627702
Random Forest – Avg Sales Change   : 174.93093166666694

Linear Regression – Std Dev: 44.78608457826838
Random Forest – Std Dev   : 102.44026096972459


In [55]:
# Tiny perturbation (5%)
X_small = X.copy()
X_small[AD_COL] = X_small[AD_COL] * 1.05

rf_small = rf_final.predict(X_small)
rf_jump = rf_small - rf_base

print("Max RF jump from small change:", np.max(np.abs(rf_jump)))


Max RF jump from small change: 252.27560000000085


In [56]:
def choose_model(lr_mae, rf_mae, lr_volatility, rf_volatility):
    if rf_mae < lr_mae and rf_volatility > lr_volatility:
        return "Reject RF: Higher accuracy but higher business risk"
    return "Accept RF"

decision = choose_model(
    lr_mae=np.mean(lr_mae),
    rf_mae=np.mean(rf_mae),
    lr_volatility=np.std(lr_delta),
    rf_volatility=np.std(rf_delta)
)

print(decision)


Accept RF


In [57]:
print("""
Even though Random Forest achieves lower prediction error,
its counterfactual instability and sensitivity to small input changes
create unacceptable business risk.

Therefore, Linear Regression is preferred for decision-making,
despite weaker accuracy.
"""
     )


Even though Random Forest achieves lower prediction error,
its counterfactual instability and sensitivity to small input changes
create unacceptable business risk.

Therefore, Linear Regression is preferred for decision-making,
despite weaker accuracy.



## Robustness and Stability Testing

In [66]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error


In [67]:
TARGET = "Weekly_Sales"
X = data.drop(columns=[TARGET])
y = data[TARGET]


In [68]:
rf = RandomForestRegressor(
    random_state=42,
    max_depth=None,
    n_jobs=-1
)

param_grid = {
    "n_estimators": [100, 200, 300]
}

tscv = TimeSeriesSplit(n_splits=5)

grid = GridSearchCV(
    rf,
    param_grid=param_grid,
    cv=tscv,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

grid.fit(X, y)

best_rf = grid.best_estimator_

print("Best n_estimators:", grid.best_params_["n_estimators"])


Best n_estimators: 300


In [69]:
baseline_preds = best_rf.predict(X)
baseline_mae = mean_absolute_error(y, baseline_preds)

print("Baseline MAE:", baseline_mae)


Baseline MAE: 42.97172933333313


In [70]:
np.random.seed(42)

drop_frac = 0.05
drop_idx = np.random.choice(
    X.index,
    size=int(len(X) * drop_frac),
    replace=False
)

X_perturbed = X.drop(index=drop_idx)
y_perturbed = y.drop(index=drop_idx)


In [71]:
noise = np.random.normal(0, 0.05, X_perturbed.shape)
X_perturbed = X_perturbed * (1 + noise)


In [72]:
rf_perturbed = RandomForestRegressor(
    n_estimators=best_rf.n_estimators,
    random_state=42,
    n_jobs=-1
)

rf_perturbed.fit(X_perturbed, y_perturbed)


RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)

In [76]:
perturbed_preds = rf_perturbed.predict(X_perturbed)
perturbed_mae = mean_absolute_error(y_perturbed, perturbed_preds)

print("Perturbed MAE:", perturbed_mae)


Perturbed MAE: 47.71272526315773


In [77]:
delta_mae = perturbed_mae - baseline_mae

print("Change in MAE (ΔMAE):", delta_mae)


Change in MAE (ΔMAE): 4.740995929824599


In [78]:
retrained_preds_full = rf_perturbed.predict(X)

prediction_instability_mae = mean_absolute_error(
    baseline_preds,
    retrained_preds_full
)

print("Prediction instability MAE:", prediction_instability_mae)


Prediction instability MAE: 66.42074444444457


In [79]:
summary = pd.DataFrame({
    "Metric": [
        "Baseline MAE",
        "Perturbed MAE",
        "Δ MAE (Robustness Loss)",
        "Prediction Instability MAE"
    ],
    "Value": [
        baseline_mae,
        perturbed_mae,
        delta_mae,
        prediction_instability_mae
    ]
})

summary


,Metric,Value
0,Baseline MAE,42.971729
1,Perturbed MAE,47.712725
2,Δ MAE (Robustness Loss),4.740996
3,Prediction Instability MAE,66.420744


In [73]:
retrained_preds_full = rf_perturbed.predict(X)

prediction_diff = retrained_preds_full - baseline_preds


In [74]:
print("Average prediction change:", np.mean(prediction_diff))
print("Std deviation of change :", np.std(prediction_diff))
print("Max absolute change     :", np.max(np.abs(prediction_diff)))


Average prediction change: -8.349075555555688
Std deviation of change : 87.73084099646363
Max absolute change     : 247.02429999999973


In [82]:
print("""
Stability Interpretation:

1. If MAE increases noticeably after small perturbations,
   the model is sensitive to data changes.

2. Large standard deviation or max prediction change
   indicates unstable behavior.

3. Such instability is a hidden business risk when:
   - Models are updated infrequently
   - Predictions drive budget decisions
""" 
"""
The Random Forest model shows a noticeable increase in MAE and
significant prediction instability after minor data perturbations.

This indicates limited robustness and stability, which poses
business risk when models are updated infrequently and used
for budget allocation decisions.
"""

     )


Stability Interpretation:

1. If MAE increases noticeably after small perturbations,
   the model is sensitive to data changes.

2. Large standard deviation or max prediction change
   indicates unstable behavior.

3. Such instability is a hidden business risk when:
   - Models are updated infrequently
   - Predictions drive budget decisions

The Random Forest model shows a noticeable increase in MAE and
significant prediction instability after minor data perturbations.

This indicates limited robustness and stability, which poses
business risk when models are updated infrequently and used
for budget allocation decisions.



## Comparision Betweem Linear Regression and Non - Parametric Model

### Stability Comparision


In [83]:
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=200, random_state=42)

lr.fit(X, y)
rf.fit(X, y)

lr_preds_base = lr.predict(X)
rf_preds_base = rf.predict(X)


In [84]:
np.random.seed(42)

# Remove 5% of rows
drop_idx = np.random.choice(X.index, size=int(0.05 * len(X)), replace=False)

X_pert = X.drop(index=drop_idx)
y_pert = y.drop(index=drop_idx)

# Add small noise (5%)
noise = np.random.normal(0, 0.05, X_pert.shape)
X_pert = X_pert * (1 + noise)


In [85]:
lr_pert = LinearRegression()
rf_pert = RandomForestRegressor(n_estimators=200, random_state=42)

lr_pert.fit(X_pert, y_pert)
rf_pert.fit(X_pert, y_pert)


RandomForestRegressor(n_estimators=200, random_state=42)

In [86]:
lr_pert = LinearRegression()
rf_pert = RandomForestRegressor(n_estimators=200, random_state=42)

lr_pert.fit(X_pert, y_pert)
rf_pert.fit(X_pert, y_pert)


RandomForestRegressor(n_estimators=200, random_state=42)

In [87]:
lr_preds_new = lr_pert.predict(X)
rf_preds_new = rf_pert.predict(X)

lr_stability_mae = mean_absolute_error(lr_preds_base, lr_preds_new)
rf_stability_mae = mean_absolute_error(rf_preds_base, rf_preds_new)

print("Linear Regression – Stability MAE:", lr_stability_mae)
print("Random Forest     – Stability MAE:", rf_stability_mae)


Linear Regression – Stability MAE: 16.762393934174817
Random Forest     – Stability MAE: 66.86839666666681


In [88]:
print("""
Linear Regression is significantly more stable than Random Forest.
Small changes in training data do not materially alter predictions.

Random Forest exhibits higher sensitivity, creating risk when
models are updated infrequently or used for budgeting decisions.
"""
     )


Linear Regression is significantly more stable than Random Forest.
Small changes in training data do not materially alter predictions.

Random Forest exhibits higher sensitivity, creating risk when
models are updated infrequently or used for budgeting decisions.



### Interpretability Comparison

In [89]:
lr_coef = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": lr.coef_
}).sort_values(by="Coefficient", ascending=False)

lr_coef


,Feature,Coefficient
2,Competitor_Price,13.372099
0,Advertising_Spend,1.899227
1,Price,-20.124950


In [90]:
rf_importance = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

rf_importance


,Feature,Importance
0,Advertising_Spend,0.579435
1,Price,0.298249
2,Competitor_Price,0.122316


In [91]:
comparison = pd.DataFrame({
    "Aspect": [
        "Predictive Stability",
        "Sensitivity to Data Changes",
        "Interpretability",
        "Counterfactual Analysis",
        "Suitability for Budget Allocation"
    ],
    "Linear Regression": [
        "High",
        "Low",
        "High (coefficients)",
        "Reliable",
        "Preferred"
    ],
    "Random Forest": [
        "Low to Moderate",
        "High",
        "Low (feature importance only)",
        "Unstable",
        "Risky"
    ]
})

comparison

,Aspect,Linear Regression,Random Forest
0,Predictive Stability,High,Low to Moderate
1,Sensitivity to Data Changes,Low,High
2,Interpretability,High (coefficients),Low (feature importance only)
3,Counterfactual Analysis,Reliable,Unstable
4,Suitability for Budget Allocation,Preferred,Risky


# Deployment recommendation for a regulated business and My defensive data

In [102]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit


In [104]:
TARGET = "Weekly_Sales"
PRIMARY_FEATURE = "Advertising_Spend"

X = data.drop(columns=[TARGET])
y = data[TARGET]


### 1️⃣ Explainability Proof (Regulatory Requirement #1) - Linear Regression → FULL explainability

In [105]:
lr = LinearRegression()
lr.fit(X, y)

lr_explain = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": lr.coef_
}).sort_values(by="Coefficient", ascending=False)

lr_explain


,Feature,Coefficient
2,Competitor_Price,13.372099
0,Advertising_Spend,1.899227
1,Price,-20.124950


In [106]:
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X, y)

rf_explain = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

rf_explain


,Feature,Importance
0,Advertising_Spend,0.579435
1,Price,0.298249
2,Competitor_Price,0.122316


In [107]:
print("""
Only Linear Regression satisfies explainability and auditability
requirements expected in regulated businesses.
"""
     )


Only Linear Regression satisfies explainability and auditability
requirements expected in regulated businesses.



### 2️⃣ Predictive Stability Proof (Quarterly Updates) - Train baseline models

In [108]:
lr_preds_base = lr.predict(X)
rf_preds_base = rf.predict(X)


In [109]:
np.random.seed(42)

drop_idx = np.random.choice(X.index, size=int(0.05 * len(X)), replace=False)

X_pert = X.drop(index=drop_idx)
y_pert = y.drop(index=drop_idx)

noise = np.random.normal(0, 0.05, X_pert.shape)
X_pert = X_pert * (1 + noise)


In [110]:
lr_pert = LinearRegression()
rf_pert = RandomForestRegressor(n_estimators=200, random_state=42)

lr_pert.fit(X_pert, y_pert)
rf_pert.fit(X_pert, y_pert)


RandomForestRegressor(n_estimators=200, random_state=42)

In [111]:
lr_stability = mean_absolute_error(lr_preds_base, lr_pert.predict(X))
rf_stability = mean_absolute_error(rf_preds_base, rf_pert.predict(X))

print("Linear Regression Stability MAE:", lr_stability)
print("Random Forest Stability MAE    :", rf_stability)


Linear Regression Stability MAE: 16.762393934174817
Random Forest Stability MAE    : 66.86839666666681


### 3️⃣ Counterfactual Safety Proof (Budget Decisions) - Counterfactual: +20% Advertising Spend

In [112]:
X_cf = X.copy()
X_cf[PRIMARY_FEATURE] *= 1.20


In [113]:
lr_cf_change = lr.predict(X_cf) - lr_preds_base
rf_cf_change = rf.predict(X_cf) - rf_preds_base


In [114]:
print("LR Avg Counterfactual Change:", np.mean(lr_cf_change))
print("RF Avg Counterfactual Change:", np.mean(rf_cf_change))

print("\nLR Counterfactual Std Dev:", np.std(lr_cf_change))
print("RF Counterfactual Std Dev:", np.std(rf_cf_change))


LR Avg Counterfactual Change: 189.6698471627702
RF Avg Counterfactual Change: 174.93093166666694

LR Counterfactual Std Dev: 44.78608457826838
RF Counterfactual Std Dev: 102.44026096972459


### 4️⃣ Model Risk Proof (Accuracy ≠ Safety) - Time-based MAE comparison

In [115]:
tscv = TimeSeriesSplit(n_splits=5)

lr_mae, rf_mae = [], []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    lr.fit(X_train, y_train)
    rf.fit(X_train, y_train)
    
    lr_mae.append(mean_absolute_error(y_test, lr.predict(X_test)))
    rf_mae.append(mean_absolute_error(y_test, rf.predict(X_test)))

print("Linear Regression MAE:", np.mean(lr_mae))
print("Random Forest MAE    :", np.mean(rf_mae))


Linear Regression MAE: 99.43916515970898
Random Forest MAE    : 124.04444679999997


In [116]:
decision = pd.DataFrame({
    "Criteria": [
        "Explainability",
        "Auditability",
        "Prediction Stability",
        "Counterfactual Safety",
        "Regulatory Risk",
        "Deployment Suitability"
    ],
    "Linear Regression": [
        "High",
        "High",
        "High",
        "Safe",
        "Low",
        "APPROVED"
    ],
    "Random Forest": [
        "Low",
        "Low",
        "Low",
        "Unsafe",
        "High",
        "REJECTED"
    ]
})

decision

,Criteria,Linear Regression,Random Forest
0,Explainability,High,Low
1,Auditability,High,Low
2,Prediction Stability,High,Low
3,Counterfactual Safety,Safe,Unsafe
4,Regulatory Risk,Low,High
5,Deployment Suitability,APPROVED,REJECTED


In [117]:
print("""
Python evidence shows that although Random Forest may achieve
higher predictive accuracy, it fails on explainability,
predictive stability, and counterfactual reliability.

Linear Regression satisfies regulatory expectations for
transparency, auditability, and decision safety.

Therefore, Linear Regression is the only model approved
for deployment in a regulated business environment.
"""
     )


Python evidence shows that although Random Forest may achieve
higher predictive accuracy, it fails on explainability,
predictive stability, and counterfactual reliability.

Linear Regression satisfies regulatory expectations for
transparency, auditability, and decision safety.

Therefore, Linear Regression is the only model approved
for deployment in a regulated business environment.

